dependencies </n>
Env :: Forecastinit
Python=3.11.5

Libraries

In [72]:
import pandas as pd
import numpy as np

Input

In [73]:
Historical_data_org = pd.read_csv('Input/Actual.csv')
TimeKey_org = pd.read_csv('Input/Time.csv')
Forecast_level = ['Version.[Version Name]', 'Channel.[Channel]', 'Account.[Account]',
       'PnL.[PnL]', 'Demand Domain.[Demand Domain]', 'Region.[Region]',
       'Location.[Location]', 'Time.[Planning Month]', 'Item.[Item]']
Time_column = ['Time.[Planning Month]']
Historical_data_column = ['Actual']
#Date should be in format of time key column, which is available in TimeKey file 
Time_key_column_name = "Time.[PlanningMonthKey]"
Time_key_date_format = "%m/%d/%Y %I:%M:%S %p"
Historic_start_date = "M09-20" 
Historic_end_date = "M12-22"
Forecast_start_date = "M01-23"
Forecast_end_date = "M03-23"


Processing Input

In [74]:
Forecast_level_minus_time = Forecast_level.copy()
Forecast_level_minus_time.remove(Time_column[0])
Forecast_level_minus_time

['Version.[Version Name]',
 'Channel.[Channel]',
 'Account.[Account]',
 'PnL.[PnL]',
 'Demand Domain.[Demand Domain]',
 'Region.[Region]',
 'Location.[Location]',
 'Item.[Item]']

Copy Input 

In [75]:
Historical_data = Historical_data_org.copy(deep=True)
TimeKey = TimeKey_org.copy(deep=True)
TimeKey['TimeKey'] = pd.to_datetime(TimeKey[Time_key_column_name], format=Time_key_date_format)

In [76]:
# calculating Historic_end_date_key
Historic_start_date_key = pd.to_datetime(TimeKey[TimeKey[Time_column[0]]==Historic_start_date]['TimeKey'].values[0])
Historic_end_date_key = pd.to_datetime(TimeKey[TimeKey[Time_column[0]]==Historic_end_date]['TimeKey'].values[0])
Forecast_start_date_key = pd.to_datetime(TimeKey[TimeKey[Time_column[0]]==Forecast_start_date]['TimeKey'].values[0])
Forecast_end_date_key = pd.to_datetime(TimeKey[TimeKey[Time_column[0]]==Forecast_end_date]['TimeKey'].values[0])

Processing Copy Input

In [77]:
Historical_data[Historical_data_column] = Historical_data[Historical_data_column].astype(float)


Creating Key in Historical Data

In [78]:
# creating keys 
Historical_data['key'] = Historical_data[Forecast_level_minus_time].astype(str).agg("__ MDJoinner__".join, axis=1)
# dropping columns, which is already present in keys 
Historical_data.drop(Forecast_level_minus_time,axis=1,inplace=True) 
Historical_data.head(2)

,Time.[Planning Month],Actual,key
0,M07-20,445.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...
1,M08-20,711.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...


Merging Time Key in Historical Data

In [79]:
Historical_data = pd.merge(Historical_data,TimeKey[Time_column+["TimeKey"]],on=Time_column,how='left')
Historical_data.head(2)

,Time.[Planning Month],Actual,key,TimeKey
0,M07-20,445.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-07-05
1,M08-20,711.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-08-02


Train Data

In [88]:
train = Historical_data[(Historical_data['TimeKey']>=Historic_start_date_key)&(Historical_data['TimeKey']<=Historic_end_date_key)]
X_train = train[['key','TimeKey']]
y_train = train[[Historical_data_column[0]]]

Test Data

In [91]:
test = Historical_data[(Historical_data['TimeKey']>=Forecast_start_date_key)&(Historical_data['TimeKey']<=Forecast_end_date_key)]
X_test = test[['key','TimeKey']]
y_test = test[[Historical_data_column[0]]]